In [ ]:
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/qwen2-7b-bnb-4bit",
    #Select from follow
    #unsloth/mistral-7b-bnb-4bit
    #unsloth/llama-2-7b-bnb-4bit
    #unsloth/qwen2-7b-bnb-4bit
    #unsloth/qwen3-14b-bnb-4bit
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)



In [ ]:
# Load dataset
dataset = load_dataset("json", data_files={"data": "nftables_alpaca_300.jsonl"}, split="data") #path to training data

# Split into train/validation/test
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_val = dataset['train']
test_set = dataset['test']

# Now split train_val into train and validation (90/10 of 80%)
train_val = train_val.train_test_split(test_size=0.1, seed=42)  # ~10% of 90% = 10% of total
train_set = train_val['train']
validation_set = train_val['test']

print(f"Train size: {len(train_set)}, Validation size: {len(validation_set)}, Test size: {len(test_set)}")

# Save test set to a JSONL file
test_set.to_json("test_set.jsonl", orient="records", lines=True)

# Format prompts
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""



In [ ]:


EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply formatting
train_set = train_set.map(formatting_prompts_func, batched=True)
validation_set = validation_set.map(formatting_prompts_func, batched=True)
test_set = test_set.map(formatting_prompts_func, batched=True)


In [ ]:

# PEFT model setup
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


In [ ]:
# Setup Trainer
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_set,
    eval_dataset=validation_set,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster if you have short sequences
    args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=1000,
        learning_rate=1e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),

)


In [ ]:
# Train the model
# Usloth requires login token for weight&bias.  Register from wandb.ai and copy login token.
trainer_stats = trainer.train()


In [ ]:
results = trainer.evaluate() # Unsloth validate function with the validation data

print("Validation Results:", results)

In [ ]:
# Save the fine-tuned model
trainer.save_model("./my_model_gwen2_nftables")

In [ ]:
# zip for download
!zip -r my_model_gwen2_nftables.zip my_model_gwen2_nftables